# Mental Health Services Final Data Prep

In [ ]:
import datetime as dt
import itertools
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, File, Schema, Table, as_table_columns
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

tqdm.pandas()

def isnum(x):
    if x is None:
        return False
    try:
        float(x)
        return True
    except ValueError:
        return False

In [ ]:
v1sid, v2sid = 'syn12181333', 'syn12181343'

v1r = pd.read_csv(syn.get(v1sid).path, parse_dates=['timestamp'])
v2r = pd.read_csv(syn.get(v2sid).path, parse_dates=['timestamp'])

v1r.head()
v2r.head()

In [ ]:
# # upload v2 to public release portal as csv
# v2p = v2r.copy()
# name = 'V2_MentalHealthSvc.csv'

# v2p.to_csv(name, index=None)
# v2p = syn.setProvenance(
#     syn.store(File(name=name, path=name, parent='syn10848316')),
#     activity=Activity(
#         name='Public Release',
#         description='Prepare data for public realease',
#         used=[v2sid],
#         executed=[
#             dict(
#                 name='IPython Notebook',
#                 url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_ImpactMHS_datafiles.ipynb'
#             )
#         ]
#     )
# )

In [ ]:
v2sid='syn12181343'

In [ ]:
v1 = v1r.drop(columns=[
    'userid', 'timestampUTC'
]).rename(columns={
    'brightenid':'participant_id',
    'timestamp':'dt_response'
})

# convert the response columns to appropiate indicators
def fx(x):
    if x == 'No':
        return 0
    elif x == 'Yes':
        return 1
    else:
        return -1
    
for i in range(1, 6):
    v1[f'mhs_{i}'] = v1[f'mhs_{i}'].apply(fx)

v1.head()

In [ ]:
v2 = v2r.rename(columns={
    'userid':'participant_id',
    'timestamp':'dt_response'
})

# convert the response columns to appropiate indicators
def fx(x):
    if x == 'No':
        return 0
    elif x == 'Yes':
        return 1
    else:
        return -1
    
for i in range(1, 6):
    v2[f'mhs_{i}'] = v2[f'mhs_{i}'].apply(fx)

v2.head()

### Combine the DataFrames

In [ ]:
combined = pd.concat([v1, v2], sort=False)
combined.head()

### Add week into study

In [ ]:
metasid = 'syn17023349'
metadata = syn.tableQuery(f'SELECT participant_id, startdate FROM {metasid}').asDataFrame(convert_to_datetime=True)
metadata.startdate = pd.to_datetime(metadata.startdate)

# add in the participants start date as a new column
combined = pd.merge(combined, metadata, on='participant_id', how='left')

# get the time difference in weeks as a float
combined['week'] = [
    d.days/7 for d in (
        combined.dt_response.apply(
            lambda x: dt.datetime(year=x.year, month=x.month, day=x.day))-combined.startdate
    )
]

# convert the week number to an int by taking the floor
combined.week = combined.week.progress_apply(lambda x: np.int16(np.floor(x))+1)# if not pd.isnull(x) else np.nan)

# remove the start date
combined = combined.drop(columns=['startdate'], errors='ignore')

# reorder the columns
cols = list(combined.columns)
cols = cols[0:2] + ['week'] + cols[2:-1]
combined = combined.reindex(columns=cols)

combined.head()

In [ ]:
combined.week.hist()

### Localize timestamps

In [ ]:
# localize timestamps
combined['dt_response'] = [
    str(t.tz_localize('UTC'))
    for t in combined.dt_response
]

combined.head()

### Set provenance and upload to Synapse

In [ ]:
t = syn.delete(
    syn.tableQuery('select * from syn17022660')
)

In [ ]:
final = syn.store(Table(
    Schema(
            name='Mental Health Services',
            columns=as_table_columns(combined), 
            parent='syn10848316'),
        combined
    )
)

In [ ]:
final = syn.setProvenance(
    'syn17022660',
    activity=Activity(
        name='Combine V1 and V2 data',
        description='Process and combine the data collected during study 1 and study 2',
        used=[v1sid, v2sid, metasid],
        executed=[
            dict(
                name='IPython Notebook',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_MentalHealthServices_datafiles.ipynb'
            )
        ]
    )
)